In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Imports

In [ ]:
# import sys
# sys.path.append('/content/drive/My Drive/emc918/Code/')

from Transformer.transformer import Transformer, Optimiser
from Transformer.output_layers import Regression_Output, Classification_Output
from Data.preprocessing import read_file, calc_moving_average, load_data, save_data, \
    get_raw_dataset, get_ma_dataset, prod_class, split_data
from Data.chart_patterns import plot_price, plot_peaks, get_peaks, head_shoulders, \
    inv_head_shoulders, broad_top, broad_bottom, triangle_top, triangle_bottom, get_cp_dataset
from rnn import RNN
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

## Load Dataset

In [ ]:
filename = "Data/S&P_500.csv"
# filename = "Data/DJIA.csv"

# filename = "/content/drive/My Drive/emc918/Code/Data/S&P_500.csv"
# filename = "/content/drive/My Drive/emc918/Code/Data/DJIA.csv"

### Regression Datasets

In [ ]:
# RAW prices
training_set, test_set = get_raw_dataset(filename)
d_model = 4
len_seq = 5
h = 4

In [ ]:
# MA prices
training_set, test_set = get_ma_dataset(filename)
d_model = 6
len_seq = 5
h = 6

### Classification Datasets

In [ ]:
# MA prices
training_set, test_set = get_ma_dataset(filename)
training_set = prod_class(training_set)
test_set = prod_class(test_set, 0)
d_model = 6
len_seq = 5
h = 6

In [ ]:
# Chart Patterns

df = read_file(filename)['Close']
training_set, test_set = split_data(df, 0.2)
test_set, val_set = split_data(test_set, 0.5)

training_set = get_cp_dataset(training_set)
test_set = get_cp_dataset(test_set)
val_set = get_cp_dataset(val_set)

d_model = 8
len_seq = 10
h = 8

### Prepare Set

In [ ]:
if torch.cuda.is_available():
    device = "cuda:0"
    for i in training_set:
        i.move_to_device(device)
    for i in test_set:
        i.move_to_device(device)
    print("Tensors moved to GPU.")
else:
    device = "cpu"

In [ ]:
for b in test_set:
    b.dec_inp = b.enc_inp

val_set = test_set[-5:]
test_set = test_set[:-5]

## Transformer

In [ ]:
N = 2
epochs = 50

In [ ]:
# Regression Model
Output_layer = Regression_Output(d_model)
model = Transformer(d_model, len_seq, embed_req=False, N=N, h=h, output_layer=Output_layer)
model.double()

In [ ]:
# Classification Model
Output_layer = Classification_Output(d_model)
model = Transformer(d_model, len_seq, embed_req=False, N=N, h=h, output_layer=Output_layer, c=True)
model.double()

In [ ]:
if torch.cuda.is_available():
    model.to(torch.device(device))
    print("Model moved to GPU.")

### Training

In [ ]:
loss, outputs, loss_test, outputs_test = [], [], [], []
for epoch in range(epochs):
    temp, temp2 = model.run_epoch(training_set)
    loss.append(temp)
    outputs.append(temp2)
    temp, temp2 = model.run_epoch(test_set, training=False)
    loss_test.append(temp)
    outputs_test.append(temp2)

validation_score, out = model.run_epoch(val_set, training=False)
print("VALIDATION ACHIEVED: ", validation_score)

In [ ]:
x = np.arange(1, epochs+1)

plt.plot(x[1:],loss[1:])
plt.show()
plt.plot(x[1:],loss_test[1:])
plt.show()

## Baseline Model

In [ ]:
epochs = 50

In [ ]:
# Regression Model
baseline = RNN(d_model, d_model)

In [ ]:
# Classification Model
baseline = RNN(d_model, 3)

In [ ]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    baseline.to(torch.device(device))
    print("Model moved to GPU.")

### Training

In [ ]:
loss, outputs, loss_test, outputs_test = [], [], [], []
for epoch in range(epochs):
    temp, temp2 = baseline.run_epoch(training_set)
    loss.append(temp)
    outputs.append(temp2)
    temp, temp2 = baseline.run_epoch(test_set, training=False)
    loss_test.append(temp)
    outputs_test.append(temp2)

validation_score, out = baseline.run_epoch(val_set, training=False)
print("VALIDATION ACHIEVED: ", validation_score)